# Single-View Geometry (Python)

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- You will need [GUI backend](https://matplotlib.org/faq/usage_faq.html#what-is-a-backend) to enable interactive plots in `matplotlib`.
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [1]:
#%matplotlib tk
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

from PIL import Image
import pickle

# Provided functions

In [2]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))

    plt.figure()
    plt.imshow(im)
    print('Set at least {} lines to compute vanishing point'.format(min_lines))
    while True:
        print('Click the two endpoints, use the right key to undo, and use the middle key to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print('Need at least {} lines, you have {} now'.format(min_lines, n))
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1

    return n, lines, centers

In [3]:
def plot_lines_and_vp(ax, im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10
    
    ax.imshow(im)
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        ax.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')

    ax.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro')
    ax.set_xlim([bx1, bx2])
    ax.set_ylim([by2, by1])

# Your implementation

In [4]:
def get_vanishing_point(lines):
    """
    Solves for the vanishing point using the user-input lines.
    """
    # <YOUR CODE>
    pt1 = np.cross(lines[:, 0], lines[:, 1])/np.cross(lines[:, 0], lines[:, 1])[-1]
    pt2 = np.cross(lines[:, 1], lines[:, 2])/np.cross(lines[:, 1], lines[:, 2])[-1]
    pt3 = np.cross(lines[:, 0], lines[:, 2])/np.cross(lines[:, 0], lines[:, 2])[-1]
    point = (pt1 + pt2 + pt3)/3
    return point

In [5]:
def get_horizon_line(vpts):
    """
    Calculates the ground horizon line.
    """
    # <YOUR CODE>
    line = np.cross(vpts[:,1], vpts[:,0])
    line = line/np.sqrt(line[0]**2 + line[1]**2)
    return line

In [6]:
def plot_horizon_line(img,horizon_line, vpts):#im, line):
    """
    Plots the horizon line.
    """
    pt1, pt2 = vpts[:, 0], vpts[:, 1]
    plt.imshow(im)
    plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')
    plt.plot(pt1[0], pt1[1], 'ro')
    plt.plot(pt2[0], pt2[1], 'ro')
    plt.title('Horizon Line')
    plt.show()
    return


In [7]:
def get_camera_parameters(vpts):
    """
    Computes the camera parameters. Hint: The SymPy package is suitable for this.
    """
    # <YOUR CODE>
    f, u, v = sp.Symbol('f'),sp.Symbol('u'),sp.Symbol('v')
    
    K = sp.Matrix(((f, 0, u), (0, f, v), (0, 0, 1))).inv()
    e1 = sp.Matrix(vpts[:, 0]).T * K.T * K* sp.Matrix(vpts[:, 1])
    e2 = sp.Matrix(vpts[:, 0]).T * K.T * K* sp.Matrix(vpts[:, 2])
    e3 = sp.Matrix(vpts[:, 1]).T * K.T * K* sp.Matrix(vpts[:, 2])
    
    sol = sp.solve([e1, e2, e3], [f, u, v])[0]
    K = sp.Matrix(((sol[0],0,sol[1]),(0,sol[0],sol[2]), (0,0,1)))
    return sol[0], sol[1], sol[2], K 

In [8]:
def get_rotation_matrix(K, vpts):
    """
    Computes the rotation matrix using the camera parameters.
    """
    
    a = np.matmul(np.array(K.inv()).astype(np.float), vpts[:, 0])
    b = np.matmul(np.array(K.inv()).astype(np.float), vpts[:, 1])
    c = np.matmul(np.array(K.inv()).astype(np.float), vpts[:, 2])
    a = (a/np.linalg.norm(a)).T.reshape(3,1)
    b = (b/np.linalg.norm(b)).T.reshape(3,1)
    c = (c/np.linalg.norm(c)).T.reshape(3,1)
    R = np.concatenate((a,b,c)).reshape(3,3).T
    return R

In [9]:
def get_homography(K,Rf):
    """
    Compute homography for transforming the image into fronto-parallel 
    views along the different axes.
    """
    # <YOUR CODE>
    
    H = K*Rf*K.inv()
    return H

In [10]:
def get_rotation_matrix_rectification(K, vpts,u,v):
    """
    Compute the rotation matrix that will be used to compute the 
    homography for rectification.
    """
    vpts[:,2] = [u,v,1]
    a = np.matmul(np.array(K.inv()).astype(np.float),vpts[:, 0])
    b = np.matmul(np.array(K.inv()).astype(np.float), vpts[:, 1])
    c = np.matmul(np.array(K.inv()).astype(np.float), vpts[:, 2])
    a = (a/np.linalg.norm(a)).T.reshape(3,1)
    b = (b/np.linalg.norm(b)).T.reshape(3,1)
    c = (c/np.linalg.norm(c)).T.reshape(3,1)
    R = np.concatenate((a,b,c)).reshape(3,3).T
    return R

# Main function

In [11]:
im = np.asarray(Image.open('./data/Q3/eceb.png'))

# Also loads the vanishing line data if it exists in data.pickle file. 
# data.pickle is written using snippet in the next cell.
if os.path.exists('data.pickle'):
    with open('data.pickle', 'rb') as f:
        all_n, all_lines, all_centers = pickle.load(f, encoding='latin')
    num_vpts = 3

In [12]:
# Part (a)
# Computing vanishing points for each of the directions
vpts = np.zeros((3, num_vpts))

for i in range(num_vpts):
    #fig = plt.figure(); ax = fig.gca()
    
    # <YOUR CODE> Solve for vanishing point
    vpts[:, i] = get_vanishing_point(all_lines[i])
    
    # Plot the lines and the vanishing point
    #plot_lines_and_vp(ax, im, all_lines[i], vpts[:, i])
    #fig.savefig('Q3_vp{:d}.pdf'.format(i), bbox_inches='tight')
print(vpts)

[[6.79783058e+02 6.28281368e+03 1.00440486e+03]
 [2.30698890e+02 1.48572302e+02 3.54061952e+03]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00]]


In [13]:
# Part (b) Computing and plotting the horizon
# <YOUR CODE> Get the ground horizon line
horizon_line = get_horizon_line(vpts)
print(horizon_line)
# # <YOUR CODE> Plot the ground horizon line
# fig = plt.figure(); ax = fig.gca()
# plot_horizon_line(im,horizon_line, vpts)#im,horizon_line)
# fig.savefig('Q3_horizon.pdf', bbox_inches='tight')

[-1.46559554e-02 -9.99892596e-01  2.40636982e+02]


In [14]:
# Part (c) Computing Camera Parameters
# <YOUR CODE> Solve for the camera parameters (f, u, v)
import sympy as sp
f, u, v, K = get_camera_parameters(vpts)
print(u, v, f, K)

962.334249004020 670.380474702450 -1128.66219621603 Matrix([[-1128.66219621603, 0, 962.334249004020], [0, -1128.66219621603, 670.380474702450], [0, 0, 1]])


In [15]:
# Part (d) Computing Rotation Matrices
# <YOUR CODE> Solve for the rotation matrix
R = get_rotation_matrix(K, vpts)
print(R)

[[ 0.22716807 -0.97376005 -0.01363952]
 [ 0.35349919  0.09550191 -0.93054699]
 [ 0.90743208  0.20656901  0.36591838]]


In [16]:
vpts[:,1]

array([6.28281368e+03, 1.48572302e+02, 1.00000000e+00])

In [71]:
# Part (e) Generating fronto-parallel warps. Compute the 
# appropriate rotation to transform the world coordinates
# such that the axis of projection becomes the world
# X, Y and Z axes respectively. Use this rotation to estimate
# a homography that will be used to compute the output view.
# Apply the homography to generate the 3 fronto-parallel
# views and save them.

#Rt = get_rotation_matrix_rectification(K,vpts, u, v)
Rt = np.array([[0,0,1],[1,0,0],[0,1,0]])
print(Rt)
Rf = np.matmul(Rt,np.linalg.inv(R))
H = get_homography(K,Rf)
print(H)

[[0 0 1]
 [1 0 0]
 [0 1 0]]
Matrix([[0.816620098084167, -1.01197504019789, -321.662989593588], [0.805542829142291, 0.296774855944356, -1859.85798162821], [0.000862755969725248, -8.46151407737849e-5, -0.566966272632479]])


In [76]:
from skimage.transform import SimilarityTransform 
from skimage.transform import warp
img = cv2.imread('./data/Q3/eceb.png', cv2.IMREAD_COLOR) 
H = np.array(H).astype(np.float64)
im_dst = cv2.warpPerspective(img, H, (300,300))
cv2.imwrite('Q3_im{:d}.jpg'.format(i), im_dst,
            [int(cv2.IMWRITE_JPEG_QUALITY), 90])

True